In [ ]:
#SETUP always run this
import math
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path = "stockcharts.csv"

allData = pd.read_csv(path)

#drop rows where sector is Nan
allData = allData.dropna(subset=['Sector'])

# Create a dictionary of all the sectors mapped to a list of all the industries in that sector mapped to a list of all the tickers in that industry
sectors = {}
for index, row in allData.iterrows():
    if row['Sector'] not in sectors:
        sectors[row['Sector']] = {}
    if row['Industry'] not in sectors[row['Sector']]:
        sectors[row['Sector']][row['Industry']] = []
    sectors[row['Sector']][row['Industry']].append(row['Ticker'])


allSectors = list(sectors.keys())
allIndustries = []
for sector in allSectors:
    allIndustries.extend(list(sectors[sector].keys()))
allIndustries = list(set(allIndustries))
allTickers = []
for sector in allSectors:
    for industry in sectors[sector]:
        allTickers.extend(sectors[sector][industry])
allTickers = list(set(allTickers))




In [61]:
import json
json.dump(sectors, open("sectors.json",'w'))

In [62]:
test = json.load(open("sectors.json",'r'))

In [64]:
test == sectors

True

In [ ]:
print(allSectors, len(allSectors))
print(allIndustries, len(allIndustries))
print(allTickers, len(allTickers))

In [60]:
#download data and save Close price
anchorData = yf.download(allTickers, start = "2020-01-01", end="2023-03-01")
anchorData.head()
closingPrices = anchorData["Close"]

[*********************100%***********************]  3390 of 3390 completed

26 Failed downloads:
- GLOP/PA: No timezone found, symbol may be delisted
- LGF/B: No timezone found, symbol may be delisted
- LGF/A: No timezone found, symbol may be delisted
- MS/PA: No timezone found, symbol may be delisted
- BRK/A: No timezone found, symbol may be delisted
- BML/PL: No timezone found, symbol may be delisted
- C/PK: No timezone found, symbol may be delisted
- WFC/PL: No timezone found, symbol may be delisted
- AKO/B: No timezone found, symbol may be delisted
- MKC/V: No timezone found, symbol may be delisted
- EBR/B: No timezone found, symbol may be delisted
- AKO/A: No timezone found, symbol may be delisted
- CDEV: No timezone found, symbol may be delisted
- CIG/C: No timezone found, symbol may be delisted
- BF/A: No timezone found, symbol may be delisted
- CWEN/A: No timezone found, symbol may be delisted
- PBR/A: No timezone found, symbol may be delisted
- NFGFF: No timezone found, symbol

In [ ]:
closingPrices.tail()

In [ ]:
closingPrices.to_csv("BreadthData.csv")

In [59]:

closingPrices = pd.read_csv("BreadthData.csv")
closingPrices.tail()

,Date,A,AA,AAL,AAN,AAON,AAP,AAPL,AAT,AAWW,...,ZNH,ZNTL,ZS,ZTO,ZTS,ZUMZ,ZUO,ZWS,ZYME,ZYXI
789,2023-02-21 00:00:00-05:00,143.419998,48.070000,15.69,13.85,77.570000,144.190002,148.479996,25.480000,101.739998,...,33.919998,18.549999,127.839996,24.920000,169.839996,24.090000,8.62,22.750000,8.57,12.61
790,2023-02-22 00:00:00-05:00,142.080002,48.980000,15.56,14.14,78.029999,143.619995,148.910004,25.740000,101.820000,...,33.880001,19.490000,132.160004,24.930000,166.429993,24.320000,8.70,22.959999,8.57,12.54
791,2023-02-23 00:00:00-05:00,143.199997,47.959999,15.87,14.15,80.370003,142.009995,149.399994,25.809999,101.779999,...,34.099998,19.219999,133.500000,24.740000,169.500000,24.580000,8.84,23.110001,8.63,12.63
792,2023-02-24 00:00:00-05:00,NaN,46.450001,15.56,14.13,81.470001,NaN,NaN,25.469999,101.750000,...,33.939999,18.660000,130.880005,24.450001,166.330002,NaN,8.62,22.969999,NaN,NaN
793,2023-02-27 00:00:00-05:00,142.179993,48.040001,15.63,14.28,81.489998,140.630005,147.919998,25.450001,101.360001,...,35.139999,18.750000,131.839996,24.200001,165.470001,23.200001,8.56,23.010000,8.38,12.83


In [ ]:
#remove date column from closing prices
closingPrices = closingPrices.drop(columns=["Date"])

In [ ]:
# For each ticker, calculate the 20,50,200 day moving average
movingAverages = {}
for ticker in closingPrices:
    movingAverages[(ticker, 20)] = closingPrices[ticker][-20::].mean()
    movingAverages[(ticker, 50)] = closingPrices[ticker][-50::].mean()
    movingAverages[(ticker, 200)] = closingPrices[ticker][-200::].mean()

In [ ]:
#display the tickers in each industry
movingAverages = {}

for sector in allSectors:
    print(sector)
    for industry in sectors[sector]:
        count = 0
        total = 0
        for ticker in sectors[sector][industry]:
            print("ticker", ticker)
            movingAverages[(ticker, 20)] = closingPrices[ticker][-20::].mean()
            movingAverages[(ticker, 50)] = closingPrices[ticker][-50::].mean()
            movingAverages[(ticker, 200)] = closingPrices[ticker][-200::].mean()



In [ ]:
MAs = [20,50,200]
counts = {}
for sector in allSectors:
    print(sector)
    for industry in sectors[sector]:
        count = {}
        total = 0
        for ticker in sectors[sector][industry]:
            total +=1
            for ma in MAs:
                if closingPrices[ticker][-1] > movingAverages[(ticker, ma)]:
                    count[ma] = count.get(ma,0)+1
        counts[industry] = count

    


In [ ]:
SectorToIndustry = {}
for i in range(len(allData)):
    sector = allData.iloc[i]["Sector"]
    industry = allData.iloc[i]["Industry"]
    if sector not in SectorToIndustry:
        SectorToIndustry[sector] = [industry]
    else:
        if industry not in SectorToIndustry[sector]:
            SectorToIndustry[sector].append(industry)

